In [1]:
# %pip install --user git+https://github.com/lukegre/all_my_code/
# %pip install --user pyseaflux
# R2-scripts can be found at https://github.com/RECCAP2-ocean/R2-scripts/

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import site 
sys.path.insert(0, site.USER_SITE)
sys.path.insert(0, '/UP_home/gregorl/git/R2-scripts')

from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
from dask.diagnostics import ProgressBar
from munch import Munch
from collections import ChainMap

import warnings
import numpy as np
import xarray as xr
import pandas as pd
import python as r2
import pyseaflux as sf
import all_my_code as amc

amc.data.set_defaults(resolution=1)

Could not find the plotting defaults in the standard location
Please load the data with a manual path defined
rc not loaded


/home/gregorl/.local/lib/python3.11/site-packages/all_my_code/files/save.py:4: AccessorRegistrationWarning: registration of accessor <class 'all_my_code.files.save.Save'> under name 'to_netcdf_with_compression' for type <class 'xarray.core.dataset.Dataset'> is overriding a preexisting attribute with the same name.
  @xr.register_dataset_accessor("to_netcdf_with_compression")
[AMC] version: version_undefined


# Load Data

In [4]:
import os
import re
import numpy as np

def get_fnames_recursive_search(basedir, include=[], exclude=[]):
    """
    Search and match file names in a directory (recursive)

    Parameters
    ----------
    basedir : str
        The directory that you'd like to search through.
    include : list of str, optional
        The string patterns that must occur in the files you're looking for.
    exclude : list of str, optional
        String patterns you would like to exclude from the filenames.

    Returns
    -------
    list of str
        A list of file names with the full path.
    """
    flist = []
    for path, subdir, files in os.walk(basedir):
        for fname in files:
            # Check if all 'include' patterns are present in the file name
            if all([p in fname for p in include]):
                # Check if any 'exclude' patterns are present in the file name
                has_excluded = [s in fname for s in exclude]
                if not any(has_excluded):
                    flist.append(os.path.join(path, fname))

    # Sort the list of file names
    flist = np.sort(flist)
    return flist


def open_reccap2_data(basedir, dim='model', **kwargs):
    """
    Open and process RECCAP-2 ocean data from files in a given directory.

    Parameters
    ----------
    basedir : str
        The base directory where the files are located.
    dim : str, optional
        The dimension along which to concatenate the datasets.
    **kwargs : keyword arguments
        Additional keyword arguments to pass to `get_fnames_recursive_search` for file searching.

    Returns
    -------
    xarray.DataArray or list of xarray.Dataset
        Processed ocean data as a DataArray if a single dataset is returned, or a list of Datasets if multiple datasets are returned.
    """
    # Search for files in the directory using the provided patterns
    flist = get_fnames_recursive_search(basedir=basedir, **kwargs)
    
    # Print the number of files found
    print(f"{len(flist)} files found")
    
    # Open the RECCAP-2 ocean data using the provided module
    ds = r2.data.open_reccap2_ocean_data(flist, rename_var_to_model=True)
    
    # If multiple datasets are returned, return them as a list
    if isinstance(ds, list):
        return ds
    
    # Process the single dataset
    da = (
        ds
        .where(lambda x: x < 1e34)
        .sel(time=slice('1985', '2018'))
        .to_array(dim=dim))
    
    return da


In [5]:
def get_reccap2_spco2():
    """
    Retrieve and process various spco2 datasets and observations from RECCAP-2 and related sources.

    Returns:
    - spco2 (xarray.DataArray): Concatenated spco2 datasets from models and observations.
    - socat (xarray.DataArray): Processed and filtered SOCAT observations.

    """
    # Ignore UserWarnings that might be raised within this block
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)

        # Models and assimilation datasets
        spco2_models = open_reccap2_data(
            basedir='./data/reccap2/models/2D_CO2',
            dim='model',
            include=['_A_', '.nc', 'spco2_'],
            exclude=['ROMS', 'v20211214', 'GOA', 'ECCO', 'OCIM'])

        spco2_romsso = open_reccap2_data(
            basedir='./data/reccap2/models/2D_CO2',
            dim='model',
            include=['_A_', '.nc', 'spco2_', 'ROMS-SouthernOcean'])

        spco2_ecco = open_reccap2_data(
            basedir='./data/reccap2/models/2D_CO2',
            dim='model',
            include=['ECCO', '_A_', '.nc', 'spco2_']) * 1e6
        
        # Load BSOSE model data and apply necessary operations
        spco2_bsose = open_reccap2_data(
            basedir='./data/reccap2/models/2D_CO2',
            dim='model',
            include=['SOSE', '.nc', 'spco2_']).where(lambda x: x!=0).dropna('lat', how='all') * 1e6

        # Data products from surface_co2 directory
        spco2_products = open_reccap2_data(
            basedir='./data/reccap2/surface_co2/',
            dim='model',
            include=['.nc', 'spco2_'],
            exclude=['RECCAP-XYT', ])
        
        # Load and preprocess Bushinsky's dataset
        spco2_bushinsky = (
            xr.open_dataset('/nfs/kryo/work/datasets/grd/ocean/2d/obs/pco2/bushinsky_2019/MPI-SOM_FFN_SOCCOMv2018.nc', drop_variables='date')
            .pipe(r2.data.conform_dataset)
            .spco2_smoothed
            .sel(time=slice('2012', '2018'))
            .load()
            .rename('SOMFFN_Bushinsky')
            .grid.lon_0E_360E()
            .expand_dims(model=['SOMFFN_Bushinsky']))

        # Load, filter, and process SOCAT gridded data
        socat = (
            amc.data.socat_gridded()
            .sel(time=slice('1985', '2018'))
            .load()
            .conform(time_center_monthly=True)
            .grid.lon_0E_360E()
            .pipe(lambda x: sf.fCO2_to_pCO2(x.fco2_ave_unwtd, x.sst_ave_unwtd)))
        
        # Concatenate all the spco2 datasets
        spco2 = xr.concat([spco2_models, spco2_romsso, spco2_bsose, spco2_ecco, spco2_products, spco2_bushinsky], 'model')
        
    return spco2, socat


In [6]:
southern_ocean_mask = amc.datasets.masks.reccap2_regions(resolution=1).open_ocean.grid.lon_0E_360E() == 5
sub_regions = amc.datasets.masks.reccap2_regions(resolution=1).southern.grid.lon_0E_360E().where(southern_ocean_mask, drop=True)

spco2, socat = get_reccap2_spco2()

14 files found
ADDED: spco2_CCSM-WHOI_A_1_gr_1958-2017_20211125.nc
ADDED: spco2_CESM-ETHZ_A_1_gr_1980-2018_v20211122.nc
ADDED: spco2_CESM-ETHZ_A_1_gr_1980-2018_v20221125.nc
ADDED: spco2_CNRM-ESM2-1_A_1_gr_1980-2018_v20211208.nc
ADDED: spco2_EC-Earth3_A_1_gr_1980-2018_v20220323.nc
ADDED: spco2_FESOM_REcoM_HR_A_1_gr_1980-2018_v20211119.nc
ADDED: spco2_FESOM_REcoM_LR_A_1_gr_1980-2018_v20211119.nc
ADDED: spco2_MOM6-Princeton_A_1_gr_1980-2018_v20220125.nc
ADDED: spco2_MPIOM-HAMOCC_A_1_GR15_1980-2019_v20220110.nc
ADDED: spco2_MRI-ESM2-1_A_1_gr_1980-2018_v20220502.nc
ADDED: spco2_NorESM-OC1.2_A_1_gr_1980-2018_v20211125.nc
ADDED: spco2_ORCA025-GEOMAR_A_1_gr_1980-2018_v20210804.nc
ADDED: spco2_ORCA1-LIM3-PISCES_A_1_gr_1980-2018_v20211215.nc
ADDED: spco2_PlankTOM12_A_1_gr_1980-2018_v20220404.nc
Trying to merge files
1 files found
ADDED: spco2_ROMS-SouthernOcean-ETHZ_A_1_gr_1980-2018_v20220630.nc
Trying to merge files
1 files found
ADDED: spco2_ECCO-Darwin_A_1_g_1995-2018_v20210712.nc
Trying to m

### Subset to the Southern Ocean and add data classes

In [11]:
with ProgressBar():
    spco2 = spco2.where(southern_ocean_mask, drop=True).chunk(model=1, time=-1, lat=-1, lon=-1).persist()
    socat = socat.where(southern_ocean_mask, drop=True).chunk(time=-1, lat=-1, lon=-1).persist()
    
data_class = Munch(
    gobm = ['CCSM_WHOI', 'CESM_ETHZ', 'CNRM_ESM2_1', 'EC_Earth3', 'FESOM_REcoM_HR', 'FESOM_REcoM_LR', 'MOM6_Princeton', 'MPIOM_HAMOCC', 
            'MRI_ESM2_1', 'NorESM_OC1_2', 'ORCA025_GEOMAR', 'ORCA1_LIM3_PISCES', 'PlankTOM12', 'ROMS_SouthernOcean_ETHZ'],
    ecco  = ['ECCO_Darwin'],
    bsose = ['BSOSE'],
    pco2_prod = ['AOML', 'CMEMS_LSCE_FFNN', 'CSIRML6', 'JMAMLR', 'JenaMLS', 'NIES_ML3', 'OceanSODAETHZ', 'MPI_SOMFFN', 'UOEX_Wat20', 'LDEO_HPD'],
    pco2_prod_bushin = ['SOMFFN_Bushinsky'])

data_class = pd.Series(dict(ChainMap(*[{v: k for v in vl} for k, vl in data_class.items()])))

spco2['data_class'] = xr.Coordinate(['model'], data_class[spco2.model.values].values)

[########################################] | 100% Completed | 103.91 ms
[########################################] | 100% Completed | 102.04 ms


### Check that data is OK (check units and missing data due to grid missmatches)

In [17]:
# check if all data 
with ProgressBar():
    spco2_space_avg = spco2.mean(['lat', 'lon']).to_series().unstack(level=0)
    time_counts = spco2_space_avg.count()
    lon_counts = spco2.mean(['time', 'lat']).to_series().unstack(level=0).count()
    
# check that there isn't missing data
assert (time_counts > 0).all()
assert (time_counts > 400).sum() > 10
assert (lon_counts == 360).all()

# check that pCO2 is in the correct range
assert (spco2_space_avg.min() > 200).all()
assert (spco2_space_avg.min() < 400).all()
assert (spco2_space_avg.max() > 300).all()
assert (spco2_space_avg.max() < 500).all()

# checks that coordinates are aligned
socat, spco2 = xr.align(socat, spco2, join='exact')

[########################################] | 100% Completed | 210.57 ms
[########################################] | 100% Completed | 211.62 ms


# Create datasets (bias, RMSE, subset to SOCAT)

In [44]:
with ProgressBar():
    # need to load and then re-persist the data to avoid error
    spco2_sampled_socat = spco2.where(socat.notnull().load(), drop=True).chunk(time=-1, lat=-1, lon=-1, model=1).persist()
    resid = (spco2 - socat).persist()
    

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    ds_full = xr.Dataset()
    ds_full['bias'] = resid.spatial.average_area_weighted().expand_dims(region=[0])
    ds_full['rmse'] = ((resid**2).spatial.average_area_weighted()**0.5).expand_dims(region=[0])
    ds_full['spco2'] = spco2.spatial.average_area_weighted().expand_dims(region=[0])
    ds_full['spco2_subset_socat'] = spco2_sampled_socat.spatial.average_area_weighted().expand_dims(region=[0])
    ds_full['spco2_socat'] = socat.spatial.average_area_weighted().expand_dims(region=[0])
    ds_full['socat_count'] = socat.count(['lat', 'lon']).compute()

    ds_regional = xr.Dataset()
    ds_regional['bias'] = resid.spatial.aggregate_region(sub_regions)
    ds_regional['rmse'] = (resid**2).spatial.aggregate_region(sub_regions)**0.5
    ds_regional['spco2'] = spco2.spatial.aggregate_region(sub_regions)
    ds_regional['spco2_subset_socat'] = spco2_sampled_socat.spatial.aggregate_region(sub_regions)
    ds_regional['spco2_socat'] = socat.spatial.aggregate_region(sub_regions)
    ds_regional['socat_count'] = socat.groupby(sub_regions.rename('region')).count().T
    
with ProgressBar():
    ds_full = ds_full.load()
    ds_regional = ds_regional.load()

[########################################] | 100% Completed | 103.30 ms
[########################################] | 100% Completed | 1.41 sms
[########################################] | 100% Completed | 918.84 ms
[########################################] | 100% Completed | 4.50 ss
[########################################] | 100% Completed | 4.05 sms


### Add metadata and save

In [45]:
ds = xr.concat([ds_full, ds_regional], 'region').assign_coords(region=['SO', 'STSS', 'SPSS', 'ICE']).astype('float32').transpose('region', 'model', 'time')
ds.attrs = dict(
    description="Data for Figure 9 in the RECCAP2 Southern Ocean chapter. ",
    creator="Luke Gregor",
    date=pd.Timestamp.today().strftime("%Y-%m-%d"))

ds.bias.attrs = dict(units='uatm', info="(pCO2 - SOCAT).mean([lat, lon])")
ds.rmse.attrs = dict(units='uatm', info="(square(pCO2 - SOCAT).mean([lat, lon]))**0.5")
ds.spco2.attrs = dict(units='uatm', info="pCO2 averaged over the relavant regions")
ds.spco2_socat.attrs = dict(units='uatm', info='pCO2 from SOCAT averaged over relevant regions')
ds.spco2_subset_socat.attrs = dict(units='uatm', info="pCO2 subset to the SOCAT locations and then averaged over the regions")

In [46]:
ds.to_netcdf('./fig09_data.nc')

In [49]:
ds.rmse.mean('time')

<xarray.DataArray 'rmse' (region: 4, model: 26)>
array([[28.75926  , 27.855427 , 29.833424 , 31.570833 , 36.941082 ,
        35.50217  , 29.89676  , 62.270275 , 31.99695  , 28.76164  ,
        25.074835 , 26.908884 , 35.787205 , 24.156918 , 48.198032 ,
        15.326186 , 16.114136 , 13.817489 , 15.731381 , 13.5336895,
        16.661892 , 15.064149 , 13.9354105, 17.123686 ,  8.590849 ,
        13.605196 ],
       [25.017838 , 27.952187 , 28.975904 , 25.361967 , 36.299488 ,
        37.53387  , 26.91243  , 41.318954 , 29.080276 , 24.834482 ,
        23.70427  , 27.410875 , 32.81261  , 23.076473 , 50.233673 ,
        13.934769 , 14.530018 , 14.198543 , 14.121633 , 11.5271635,
        14.673983 , 14.134984 , 11.955145 , 16.510279 ,  7.791007 ,
        10.665174 ],
       [21.5578   , 22.374392 , 22.93207  , 21.823277 , 25.714941 ,
        25.244146 , 23.451897 , 62.4578   , 23.007862 , 20.972128 ,
        20.64744  , 20.264408 , 27.247572 , 18.246801 , 32.1314   ,
        10.594884 , 11.966719 , 10.0292635, 13.236428 ,  9.847635 ,
        13.020294 , 11.548599 , 11.15496  , 13.19915  ,  7.33877  ,
        11.560179 ],
       [40.68262  , 39.790306 , 40.60534  , 50.379745 , 47.99277  ,
        42.600113 , 45.63298  , 84.37503  , 51.81338  , 48.720688 ,
        34.64161  , 34.6394   , 55.157722 , 38.714287 , 64.50731  ,
        23.867315 , 23.106234 , 17.77161  , 25.127577 , 19.907154 ,
        24.008911 , 20.758978 , 21.04922  , 22.654472 , 11.481706 ,
        20.981833 ]], dtype=float32)
Coordinates:
  * region      (region) <U4 'SO' 'STSS' 'SPSS' 'ICE'
  * model       (model) object 'CCSM_WHOI' 'CESM_ETHZ' ... 'SOMFFN_Bushinsky'
    data_class  (model) object 'gobm' 'gobm' ... 'pco2_prod' 'pco2_prod_bushin'